Some words

In [1]:
import os

import numpy as np
import pandas as pd

from tti_explorer import config
from tti_explorer.case import simulate_case, CaseFactors
from tti_explorer.contacts import EmpiricalContactsSimulator
from tti_explorer.strategies import TTIFlowModel

In [2]:
def print_doc(func):
    print(func.__doc__)

Before we do anything, let's make a random state

In [3]:
rng = np.random.RandomState(0)

We will first do a short tour of the functionality, then show how this is put together to generate simulation results.

## Generate a case

The function we use for this is `simulate_case` in `case.py`

In [4]:
print_doc(simulate_case)

simulate_case

    Args:
        rng (np.random.RandomState): random number generator.
        p_under18 (float): Probability of case being under 18
        infection_proportions (dict): Probs of being symp covid neg, symp covid pos, asymp covid pos
                                      The only required key is 'dist', which contains list of the named probs, in that order.
        p_day_noticed_symptoms (np.array[float]): Distribution of day on which case notices
            their symptoms. (In our model this is same as reporting symptoms.)
            Conditional on being symptomatic.
        inf_profile (list[float]): Distribution of initial exposure of positive secondary cases
            relative to start of primary case's infectious period.

    Returns (Case): case with attributes populated.
    


We store our config values in `config.py`. You can retrieve them as follows

In [5]:
case_config = config.get_case_config("delve")
case_config.keys()

dict_keys(['p_under18', 'infection_proportions', 'p_day_noticed_symptoms', 'inf_profile'])

We use these parameters to simulate a case

In [6]:
primary_case = simulate_case(rng, **case_config)
print_doc(primary_case)

Case(under18: bool, covid: bool, symptomatic: bool, day_noticed_symptoms: int, inf_profile: list)


Returned is a `case.Case` with stochastically generated attributes.

In [7]:
rng = np.random.RandomState(0)

### Deeper: Case attributes

Let's go through the simulated attributes of a `case.Case`. The attributes `.under18`, `.covid` and `.symptomatic` are `bool` types indicating whether the generated `case.Case` is under 18, COVID positive and symptomatic respectively. All possible values of these attributes are possible apart from the combination `.covid = False` and `.symptomatic = False` (a configuration irrelevant for the purpose of simulating infection spread). The primary case we just simulated is over 18, COVID negative but has COVID-like symptoms: 

In [8]:
print(f'Under 18: {primary_case.under18}.')
print(f'COVID positive: {primary_case.covid}.')
print(f'Symptomatic: {primary_case.symptomatic}.')

Under 18: False.
COVID positive: False.
Symptomatic: True.


Each `case.Case` also has an attribute `.day_noticed_symptoms` of type `int`, indicating the number of days from start of infectiousness until the `case.Case` noticed the symptoms. If a `case.Case` is asymptomatic, the attribute `.day_noticed_symptoms` is set to `-1`.

In [9]:
print(f'primary_case noticed symptoms {primary_case.day_noticed_symptoms} days after start of infectiousness.')

primary_case noticed symptoms 3 days after start of infectiousness.


Finally, the attribute `.inf_profile` is a `list` describing the relative infectiousness of the case for each day of the infectious period. For `primary_case`, this is `0` throughout, since `primary_case.covid = False`.

In [10]:
print(f'inf_profile is: {primary_case.inf_profile}')

inf_profile is: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


As mentioned above, the configuration for simulating these attributes are stored in `config.py`. This includes the distributions used for sampling attributes. For instance, the attribute `.under18` is sampled from a Bernoulli distribution with probability `0.21`: 

In [11]:
print(f'Probability of case being under 18: {case_config["p_under18"]}')

Probability of case being under 18: 0.21


As another example, if `case.Case` is symptomatic, the attribute `.days_noticed_symptoms` is sampled from a categorical distribution over the set {0, 1, ..., 9} (since we model an infection period of ten days in this configuration) with probabilities:

In [12]:
print(f'Probability distribution of .day_noticed_symptoms: {case_config["p_day_noticed_symptoms"]}')

Probability distribution of .day_noticed_symptoms: [0, 0.25, 0.25, 0.2, 0.1, 0.05, 0.05, 0.05, 0.05, 0.0]


## Generate contacts
Social contacts are represented by `Contacts` and defined in `contacts.py`.

To simulate social contacts, we use the BBC Pandemic Dataset. This is stratified as over/under 18 to give different patterns of social contact depending on the age of the case. 

In [13]:
def load_csv(pth):
    return np.loadtxt(pth, dtype=int, skiprows=1, delimiter=",")

In [14]:
path_to_bbc_data = os.path.join(os.pardir, "data", "bbc-pandemic")

over18 = load_csv(os.path.join(path_to_bbc_data, "contact_distributions_o18.csv"))
under18 = load_csv(os.path.join(path_to_bbc_data, "contact_distributions_u18.csv"))

Now that we have the data loaded, we use `EmpiricalContactsSimulator` to sample these tables for contacts of the primary case, then simulate their infection under a no measures scenario (i.e. no government intervention)

In [15]:
print_doc(EmpiricalContactsSimulator.__init__)

Simulate social contact using the BBC Pandemic dataset

            Each row in input arrays consists of three numbers,
            represeting number of contacts at: home, work, other

        Args:
            over18 (np.array[int], Nx3): Contact data for over 18s.
            under18 (np.array[int], Nx3): Contact data for under 18s.
            rng (np.random.RandomState): Random state.

        


In [16]:
simulate_contacts = EmpiricalContactsSimulator(over18, under18, rng)

We can now use the callable `simulate_contacts` to simulate social contacts of the primary case

In [17]:
print_doc(simulate_contacts.__call__)

Generate a social contact for the given case.

        A row from the table corresponding to the age of the `case` is sampled
        uniformly at random. A contact is generated with daily contacts as
        given by that row. These contacts are infected at random with attack rates
        given by the SARs and whether or not the `case` is symptomatic. If the
        `case` is COVID negative, then no contacts are infected.

        Args:
            case (Case): Primary case.
            home_sar (float): Secondary attack rate for household contacts.
                              (Marginal probability of infection over the whole simulation)
            work_sar (float): Secondary attack rate for contacts in the work category.
            other_sar (float): Secondary attack rate for contacts in the other category.
            asymp_factor (float): Factor by which to multiply the probabilty of secondary
                                  infection if `case` is asymptomatic COVID positive

To do this we need some more parameters, which we also load from `config.py`. The user can, of course, specify this themselves if they would like.

In [18]:
contacts_config = config.get_contacts_config("delve")

In [19]:
contacts_config.keys()

dict_keys(['home_sar', 'work_sar', 'other_sar', 'period', 'asymp_factor'])

We now do the same as we did with when simulating a primary case.

In [20]:
social_contacts = simulate_contacts(primary_case, **contacts_config)
print_doc(social_contacts)

Contacts(n_daily: dict, home: <built-in function array>, work: <built-in function array>, other: <built-in function array>)


### Deeper: Contacts attributes

Let's examine the attributes of `social_contacts`, which is an instance of `contacts.Contacts`. Note that `social_contacts` is simulated above by calling `simulate_contacts` which takes `primary_case` as in argument, so contact generation of course depends on the case simulated first. 

The first attribute to note is `.n_daily`, which is a `dict` containing the average number of daily contacts (split into three categories) of the case. This is simulated by sampling one row of the tables `over18` or `under18` depending on the value of `primary_case.under18`. In the case of `primary_case`, we can look at `social_contacts.n_daily`:

In [21]:
print(f'Average number of daily contacts for primary_case:')
print(f'Home: {social_contacts.n_daily["home"]}')
print(f'Work: {social_contacts.n_daily["work"]}')
print(f'Other: {social_contacts.n_daily["other"]}')

Average number of daily contacts for primary_case:
Home: 3
Work: 2
Other: 5


The three remaining attributes `.home`, `.work` and `.other` are arrays containing information about each contact made by the case, with one row per contact. More specifically, for each contact, the row contains the first day (always measured relative to the start of infectiousness) of encounter between the case and contact and, if transmission occurred, then the day of transmission. 

Also, recall that home contacts are assumed to repeat every day of the infectious period, whereas work/other contacts are new for each day. This means the lengths of the arrays `.work` and `.other` are `10 * .n_daily['work']` and `10 * .n_daily['other']` respectively (recalling the infection period is assumed to last ten days, a parameter set in `contacts_config['period']`). Whereas, the length of the `.home` array is just `.n_daily['home']`.

In [22]:
print(f'Lengths of .home, .work and .other attributes:')
print(f'Home: {len(social_contacts.home)}')
print(f'Work: {len(social_contacts.work)}')
print(f'Other: {len(social_contacts.other)}')

Lengths of .home, .work and .other attributes:
Home: 3
Work: 20
Other: 50


Digging further into the array, each row contains two integers. The first integer indicates the day of transmission, which is set to `-1` if no transmission occurred. The second integer contains the day of first encounter. So for instance, looking at one of the home contacts, we see transmission didn't occur and the day of first encounter is `0`, i.e. the first day of the infection period:

In [23]:
print(social_contacts.home[0])

[-1  0]


Looking at the first six work contacts, we see none of them were infected either. This is consistent with the fact that `primary_case.covid = False` so no transmission can occur in this case.

In [24]:
print(social_contacts.work[:6])

[[-1  0]
 [-1  0]
 [-1  1]
 [-1  1]
 [-1  2]
 [-1  2]]


In simulations where `case.Case` is COVID positive, each contact may get infected and the probability of getting infected depends on parameters such as the secondary attack rates (SARs), all of which are set in `contacts_config`. For details on the precise simulation procedure used to generate `contacts.Contacts`, see either Appendix A of the report or the `__call__` method of `EmpiricalContactsSimulator`.

_todo SZ: explaining the simulation procedure exactly seems overkill here. referring reader to Appendix A of the report?_

All of the information about the primary case's infection and how they infect their social contacts (under no government intervention) is now contained in `primary_case` and `social_contacts`.

... Now we run a simulation, some explanation of how that works

## TTI Strategies

## Running a Simulation

In [25]:
from tti_explorer.strategies import TTIFlowModel

In [26]:
# this getitem shouldn't have to be done, we should change this.
policy_config = config.get_strategy_configs("delve", "S0")['S0']  
factor_config = config.pop_case_factors(policy_config)

In [27]:
case_factors = CaseFactors.simulate_from(rng, primary_case, **factor_config)
tti_model = TTIFlowModel(rng, **policy_config)
metrics = tti_model(primary_case, social_contacts, case_factors)

In [28]:
pd.Series(metrics).to_frame(name="Result")

,Result
Base R,NaN
Effective R,NaN
# Manual Traces,0
# App Traces,0
# Tests Needed,0
# PersonDays Quarantined,0
Has Covid,False
Is Symptomatic,True
Got tested,False
# Secondary Infections,0
